#### Deriving New Variables

In [1]:
# Import libraries & Declare the path to the data files
import pandas as pd
import numpy as np
import os
path_data = r'C:\Users\PC\Desktop\CareerFoundry Data Analytics\08-2025 Instacart Basket Analysis'

In [2]:
# Import data 
ords_prods_merged = pd.read_pickle(os.path.join(path_data, '02 Data', 'Prepared Data','ords_prods_merged.pkl'))

In [3]:
# Create a subset (slice) to avoid memory issues
df = ords_prods_merged[:1000000]

In [4]:
#If the item’s price is lower than or equal to $5, it will be labeled a “low-range product.”
#If the item’s price is above $5 but lower than or equal to $15, it will be labeled a “mid-range product.”
#If the item’s price is above $15, it will be labeled a “high-range product.”

In [5]:
# Define the function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [6]:
# Apply the function
df['price_range'] = df.apply(price_label, axis=1)

C:\Users\PC\AppData\Local\Temp\ipykernel_12772\902492192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [7]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    673183
Low-range product    314392
High range            12425
Name: count, dtype: int64

In [8]:
# Determine the most expensive product
df['prices'].max()

99999.0

In [9]:
## Test the results using the loc() function 

In [10]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

C:\Users\PC\AppData\Local\Temp\ipykernel_12772\1169838859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [11]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [12]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [13]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    673183
Low-range product    314392
High range            12425
Name: count, dtype: int64

The if statement and the loc() function give the same results. 

In [14]:
## For - loops

In [15]:
ords_prods_merged['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [16]:
# Create a new column, “busiest day,” that will contain one of three different values: “Busiest day,” “Least busy,” and “Regularly busy.”

result = []
                        
for value in ords_prods_merged["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [17]:
ords_prods_merged['busiest_day'] = result 

In [18]:
ords_prods_merged['busiest_day'].value_counts(dropna = False) 

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

In [19]:
ords_prods_merged.shape

(32404859, 19)

1. Create a new column for the 2 busiest and 2 least busy days

I will use an if–else function to create the new column. The values for the two busiest days (0 and 1) will be assigned to the first condition, while the least busy days (3 and 4) will be assigned to the second condition. The new column will be named how_busy. The column will be added to the df. 

In [20]:
result2 = []
                        
for value in ords_prods_merged["orders_day_of_week"]:
  if value == 0 or value == 1:
    result2.append("Busy day")
  elif value == 4 or value == 3:
    result2.append("Less busy")
  else:
    result2.append("Regularly busy")

In [21]:
ords_prods_merged['how_busy'] = result2 

In [22]:
ords_prods_merged['how_busy'].value_counts(dropna = False)

how_busy
Regularly busy    12916111
Busy day          11864412
Less busy          7624336
Name: count, dtype: int64

The count of the busy days (0 and 1), 11864412 in the new column matches their combined count in the orders_day_of_week column:
0    6204182
1    5660230
The same applies to the less busy and regularly busy days. 

2. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day“. Results must be printed in a new column. 

Let’s start with an analysis of the orders per hour to determine the appropriate time slots.

In [23]:
ords_prods_merged['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

The orders can be grouped as follows:
0–7 - Fewest orders
8–19 - Most orders
20–23 - Average orders

I will create a new list called how_many_orders_per_hour and fill it by looping through each value in the column order_hour_of_day.
If the hour is between 0 and 7, the label “Fewest orders” is added.
If the hour is between 8 and 19, the label “Most orders” is added.
If the hour is between 20 and 23, the label “Average orders” is added.

The new list is assigned as a new column in the dataframe called busiest_period_of_day, so that each order is categorized according to the time of day it was placed.

In [24]:
how_many_orders_per_hour = []
                        
for value in ords_prods_merged["order_hour_of_day"]:
  if value <= 7:
    how_many_orders_per_hour.append("Fewest orders")
  elif value >= 8 and value <= 19:
    how_many_orders_per_hour.append("Most orders")
  else:
    how_many_orders_per_hour.append("Average orders")

ords_prods_merged['busiest_period_of_day'] = how_many_orders_per_hour 

In [25]:
ords_prods_merged['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       27818650
Average orders     2808334
Fewest orders      1777875
Name: count, dtype: int64

Export the dataframe in a pkl format

In [26]:
ords_prods_merged.to_pickle(os.path.join(path_data, '02 Data','Prepared Data', 'ords_prods_merged_busiest_days_and_hours.pkl'))